In [34]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from torch.utils.data import DataLoader, Dataset
from multiprocessing import Pool
from functools import partial
from multiprocessing.pool import ThreadPool

In [24]:
# Tokenizer for Seq2SQL
tokenizer = T5Tokenizer.from_pretrained('t5-base')

In [38]:
def tokenize_data(item):
    question, query = item
    input_ids = tokenizer.encode(question, truncation=True, padding='max_length', return_tensors="pt")
    label_ids = tokenizer.encode(query, truncation=True, padding='max_length', return_tensors="pt")
    return input_ids[0], label_ids[0]

In [40]:
# Custom Dataset Class
class SQLDataset(Dataset):
    def __init__(self, questions, queries, tokenizer, max_length=512):
        self.questions = questions
        self.queries = queries
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        query = self.queries[idx]
        
        # Tokenize and convert to tensors
        question_encodings = self.tokenizer.encode_plus(
            question, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt"
        )
        query_encodings = self.tokenizer.encode_plus(
            query, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt"
        )
        
        return question_encodings["input_ids"].squeeze(), query_encodings["input_ids"].squeeze()


In [41]:
if __name__ == "__main__":
    # Load the cleaned datasets
    train_dataset_path = "../data/cleaned_data/cleaned_train_dataset.csv"  # Update with your path
    validation_dataset_path = "../data/cleaned_data/cleaned_validation_dataset.csv"

    train_df = pd.read_csv(train_dataset_path)
    validation_df = pd.read_csv(validation_dataset_path)

    # Prepare the data
    questions = train_df['question'].tolist()
    queries = train_df['query'].tolist()

    # Initialize DataLoader with the updated SQLDataset
    train_dataset = SQLDataset(questions, queries, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

    # Define the Seq2SQL model
    model = T5ForConditionalGeneration.from_pretrained('t5-base')
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    model.train()

    # Training the model
    num_epochs = 3  # Set your number of epochs
    for epoch in range(num_epochs):
        for batch in train_loader:
            questions, queries = batch  # Each is now a tensor
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(input_ids=questions, labels=queries)
            
            # Compute loss
            loss = outputs.loss
            print(f'Epoch: {epoch}, Loss: {loss.item()}')
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

    # Create directory for saving the model
    model_save_dir = "../Models/seq2sql_model"
    os.makedirs(model_save_dir, exist_ok=True)

    # Save the trained model
    model.save_pretrained(model_save_dir)
    tokenizer.save_pretrained(model_save_dir)

/Users/manasmaskar/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Epoch: 0, Loss: 11.202656745910645
Epoch: 0, Loss: 4.101990222930908
Epoch: 0, Loss: 2.327583074569702
Epoch: 0, Loss: 1.8115757703781128
Epoch: 0, Loss: 1.5076320171356201
Epoch: 0, Loss: 1.5582963228225708
Epoch: 0, Loss: 1.3552078008651733
Epoch: 0, Loss: 1.3715909719467163
Epoch: 0, Loss: 1.6009360551834106
Epoch: 0, Loss: 1.2804819345474243
Epoch: 0, Loss: 1.128474473953247
Epoch: 0, Loss: 0.9469460248947144
Epoch: 0, Loss: 0.9332171082496643
Epoch: 0, Loss: 1.220102310180664
Epoch: 0, Loss: 0.8259848356246948
Epoch: 0, Loss: 0.7255359292030334
Epoch: 0, Loss: 0.7607753872871399
Epoch: 0, Loss: 0.6331817507743835
Epoch: 0, Loss: 0.7283965349197388
Epoch: 0, Loss: 0.6691262722015381
Epoch: 0, Loss: 0.573510468006134
Epoch: 0, Loss: 0.5922947525978088
Epoch: 0, Loss: 0.4901600480079651
Epoch: 0, Loss: 0.4828447997570038
Epoch: 0, Loss: 0.42231664061546326
Epoch: 0, Loss: 0.5467911958694458
Epoch: 0, Loss: 0.4278581738471985
Epoch: 0, Loss: 0.4160709083080292
Epoch: 0, Loss: 0.431968

KeyboardInterrupt: 

In [42]:
import os
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Define the model save directory
model_save_dir = "../Models/seq2sql_model"  # Update this path as needed

# Initialize the model and tokenizer (ensure you have the same architecture as before)
model = T5ForConditionalGeneration.from_pretrained('t5-base')  # Ensure this matches your training setup
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Save the trained model and tokenizer
os.makedirs(model_save_dir, exist_ok=True)  # Create the directory if it doesn't exist
model.save_pretrained(model_save_dir)
tokenizer.save_pretrained(model_save_dir)

print("Model and tokenizer saved successfully.")


/Users/manasmaskar/anaconda3/lib/python3.11/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Model and tokenizer saved successfully.


In [45]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the saved model and tokenizer
model_save_dir = "../Models/seq2sql_model"  # Update with your path
model = T5ForConditionalGeneration.from_pretrained(model_save_dir)
tokenizer = T5Tokenizer.from_pretrained(model_save_dir)

# Define 5 sample input questions
sample_questions = [
    "What is the name of the employees who work in the sales department?",
    "How many customers made a purchase last month?",
    "List all products that are currently in stock.",
    "How much revenue did the business owner, Manas generate",
    "Show the names of customers who have never made a purchase."
]

# Function to generate SQL query from a question
def generate_sql(question):
    input_ids = tokenizer.encode(question, return_tensors="pt")
    output_ids = model.generate(input_ids)
    sql_query = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return sql_query

# Evaluate the model on the sample questions
predicted_queries = [generate_sql(q) for q in sample_questions]

# Print the results
for question, predicted in zip(sample_questions, predicted_queries):
    print(f"Question: {question}")
    print(f"Predicted SQL: {predicted}")
    print("-" * 50)


Question: What is the name of the employees who work in the sales department?
Predicted SQL: What is the name of the employees who work in the sales department?
--------------------------------------------------
Question: How many customers made a purchase last month?
Predicted SQL: How many customers made a purchase last month?
--------------------------------------------------
Question: List all products that are currently in stock.
Predicted SQL: All products that are currently in stock.
--------------------------------------------------
Question: How much revenue did the business owner, Manas generate
Predicted SQL: , Manas generate?, Manas generates? owner, Man
--------------------------------------------------
Question: Show the names of customers who have never made a purchase.
Predicted SQL: Show the names of customers who have never made a purchase. Show the names of customers
--------------------------------------------------
